In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import PowerTransformer
from scipy.stats import boxcox
from collections import defaultdict
from PIL import Image
from tqdm import tqdm
import cv2
import glob
from glob import glob

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.width', None)

In [ ]:
TRAIN_IMAGE_FOLDER = "/kaggle/input/csiro-biomass/train"
TEST_IMAGE_FOLDER = "/kaggle/input/csiro-biomass/test"

In [ ]:
train_csv = pd.read_csv("/kaggle/input/csiro-biomass/train.csv")
test_csv = pd.read_csv("/kaggle/input/csiro-biomass/test.csv")

In [ ]:
print("train_csv_shape: ", train_csv.shape)
print("test_csv_shape: ", test_csv.shape)

In [ ]:
train_csv.head(10)

In [ ]:
test_csv.head(10)

In [ ]:
# Convert Sampling_Date to datetime
train_csv["Sampling_Date"] = pd.to_datetime(train_csv["Sampling_Date"], errors="coerce")

In [ ]:
def summarize_dataframe(df):
    summary = []

    for col in df.columns:
        data = df[col]
        dtype = data.dtype

        # Missing values
        missing = data.isna().sum()
        missing_pct = (missing / len(df)) * 100

        # Unique values
        unique_vals = data.nunique()

        # Initialize stats
        mean = median = mode = std = np.nan

        # Compute only for numeric columns
        if np.issubdtype(dtype, np.number):
            mean = data.mean()
            median = data.median()
            mode = data.mode().iloc[0] if not data.mode().empty else np.nan
            std = data.std()

        summary.append({
            "Column": col,
            "Dtype": dtype,
            "Missing": missing,
            "Missing_%": round(missing_pct, 2),
            "Mean": round(mean, 3) if pd.notna(mean) else "-",
            "Median": round(median, 3) if pd.notna(median) else "-",
            "Mode": round(mode, 3) if pd.notna(mode) else "-",
            "Std": round(std, 3) if pd.notna(std) else "-",
            "Unique": unique_vals
        })

    return pd.DataFrame(summary)

# 🧠 Usage:
summary_df = summarize_dataframe(train_csv)
summary_df.head(20)

In [ ]:
# State distribution
train_csv["State"].value_counts().plot.pie(autopct='%1.1f%%', figsize=(5,5), title="State Distribution")
plt.show()

In [ ]:
# Target name distribution
train_csv["target_name"].value_counts().plot.pie(autopct='%1.1f%%', figsize=(6,6), title="Target Name Distribution")
plt.show()

In [ ]:
species_counts = train_csv["Species"].value_counts()

# -----------------------------
# 1️⃣ Bar plot (frequency)
# -----------------------------
plt.figure(figsize=(8,5))
species_counts.plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Species Frequency")
plt.xlabel("Species")
plt.ylabel("Count")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:

# Sampling date histogram
plt.figure(figsize=(10,5))
sns.histplot(train_csv["Sampling_Date"], bins=30)
plt.title("Sampling Date Distribution")
plt.show()

In [ ]:
# Pre_GSHH_NDVI histogram
plt.figure(figsize=(10,5))
sns.histplot(train_csv["Pre_GSHH_NDVI"], bins=30, color="skyblue")
plt.title("Pre_GSHH_NDVI Distribution")
plt.show()


In [ ]:
# Height_Ave_cm histogram
plt.figure(figsize=(10,5))
sns.histplot(train_csv["Height_Ave_cm"], bins=20, color="orange")
plt.title("Height_Ave_cm Distribution")
plt.show()


In [ ]:
# 1️⃣ Distribution of target for each target_name
# ----------------------------
plt.figure(figsize=(10,6))
sns.boxplot(x="target_name", y="target", data=train_csv)
plt.title("Distribution of Target by Target Name")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
target_names = train_csv["target_name"].unique()

# Set up subplots — one per target_name
plt.figure(figsize=(12, 8))

for i, name in enumerate(target_names, 1):
    plt.subplot(2, (len(target_names) + 1)//2, i)  # Auto layout
    sns.histplot(train_csv[train_csv["target_name"] == name]["target"], bins=15, kde=True)
    plt.title(f"{name} - Target Distribution")
    plt.xlabel("Target")
    plt.ylabel("Frequency")

plt.tight_layout()
plt.show()

In [ ]:
from sklearn.preprocessing import PowerTransformer
from scipy.stats import boxcox, skew, kurtosis

# Setup
pt = PowerTransformer(method='yeo-johnson')
epsilon = 1e-9
target_names = train_csv["target_name"].unique()

for name in target_names:
    subset = train_csv.loc[train_csv["target_name"] == name, "target"].dropna()

    # Prepare figure
    fig, axes = plt.subplots(3, 2, figsize=(11, 10))
    fig.suptitle(f"{name} - Transformations (with Normality Stats)", fontsize=14, y=1.02)

    transforms = {
        "Log Transform": np.log1p(subset),
        "Sqrt Transform": np.sqrt(subset),
        "Cube Root Transform": np.cbrt(subset),
        "Yeo–Johnson Transform": pt.fit_transform(subset.values.reshape(-1, 1)).flatten(),
    }

    # Add Box–Cox safely
    if (subset + epsilon > 0).all():
        bc, lam = boxcox(subset + epsilon)
        transforms[f"Box–Cox Transform (λ={lam:.2f})"] = bc

    # Plot each transform
    for i, (title, data) in enumerate(transforms.items()):
        ax = axes.flat[i]
        sns.histplot(data, bins=20, kde=True, ax=ax, color="skyblue", edgecolor="black")
        ax.set_title(title)

        # Calculate skewness & kurtosis
        sk = skew(data)
        ku = kurtosis(data)
        ax.text(0.02, 0.95, f"Skew: {sk:.2f}\nKurt: {ku:.2f}",
                transform=ax.transAxes, fontsize=10,
                verticalalignment='top',
                bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray'))

    # Hide any extra subplot
    for j in range(i + 1, len(axes.flat)):
        axes.flat[j].set_visible(False)

    plt.tight_layout()
    plt.show()


In [ ]:
pt = PowerTransformer(method='yeo-johnson')
epsilon = 1e-9

# Extract the full target column
target = train_csv["target"].dropna()

# Store all transformations
transforms = {
    "Original": target,
    "Log Transform": np.log1p(target),
    "Sqrt Transform": np.sqrt(target),
    "Cube Root Transform": np.cbrt(target),
    "Yeo–Johnson Transform": pt.fit_transform(target.values.reshape(-1, 1)).flatten()
}

# Box–Cox (only if all values > 0)
if (target + epsilon > 0).all():
    bc, lam = boxcox(target + epsilon)
    transforms[f"Box–Cox Transform (λ={lam:.2f})"] = bc

# Plot all transforms with KDE + Normality stats
fig, axes = plt.subplots(3, 2, figsize=(12, 10))
fig.suptitle("Overall Target - Transformations with Normality Metrics", fontsize=15, y=1.02)

for i, (title, data) in enumerate(transforms.items()):
    ax = axes.flat[i]
    sns.histplot(data, bins=25, kde=True, ax=ax, color="skyblue", edgecolor="black")
    ax.set_title(title)

    # Compute normality statistics
    sk = skew(data)
    ku = kurtosis(data)
    ax.text(0.02, 0.95, f"Skew: {sk:.2f}\nKurt: {ku:.2f}",
            transform=ax.transAxes, fontsize=10,
            verticalalignment='top',
            bbox=dict(facecolor='white', alpha=0.7, edgecolor='gray'))

# Hide any extra subplot
for j in range(i + 1, len(axes.flat)):
    axes.flat[j].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# NOTE - 
# For Dry_Clover_g - we can do cuberoot
# For Dry_Dead_g - sqrt transform
# For Dry_Green_g - sqrt transform
# For Dry_Total_g - yeo johnson transform
# For GDM_g - box cox transform with lamda = 0.25
# For overall - sqrt transform

In [ ]:
def get_unique_sizes(directory):
    size_counts = defaultdict(int)
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', 'JPG')):
                try:
                    with Image.open(os.path.join(root, file)) as img:
                        size = img.size  # (width, height)
                        channels = len(img.getbands())  # count color channels
                        key = (size[0], size[1], channels)
                        size_counts[key] += 1
                except Exception as e:
                    print(f"Error {file}: {e}")
    return size_counts

# List of folders to check
folders = [
    TRAIN_IMAGE_FOLDER,
    TEST_IMAGE_FOLDER
]

# Display results
for folder in folders:
    print(f"\n📂 Folder: {folder}")
    sizes = get_unique_sizes(folder)

    if not sizes:
        print("⚠️ No images found or mistake in path/code")
        continue
    
    sorted_sizes = sorted(sizes.items(), key=lambda x: x[1], reverse=True)

    print("┌──────────────┬──────────────┬────────────┬─────────┐")
    print("│ Width (px)  │ Height (px) │ Channels  │ Quantity│")
    print("├──────────────┼──────────────┼────────────┼─────────┤")
    for (w, h, c), count in sorted_sizes:
        print(f"│ {w:<12} │ {h:<12} │ {c:<10} │ {count:<7} │")
    print("└──────────────┴──────────────┴────────────┴─────────┘")

In [ ]:
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

def plot_channel_histograms(image_folder, sample_limit=None, batch_size=100):
    """
    Efficiently plot pixel intensity histograms for each RGB channel across all images.
    Uses precomputed histograms (no pixel expansion, no sampling, low memory).
    """
    # List all image files
    image_files = [
        os.path.join(image_folder, f)
        for f in os.listdir(image_folder)
        if f.lower().endswith(('.jpg', '.jpeg', '.png'))
    ]

    if sample_limit:
        image_files = image_files[:sample_limit]

    print(f"📸 Found {len(image_files)} images in {image_folder}")

    # Initialize histograms (bins for pixel intensities 0–255)
    hist_r = np.zeros(256, dtype=np.int64)
    hist_g = np.zeros(256, dtype=np.int64)
    hist_b = np.zeros(256, dtype=np.int64)

    # Process in batches
    for i in tqdm(range(0, len(image_files), batch_size), desc="Processing images"):
        batch = image_files[i:i + batch_size]
        for path in batch:
            try:
                with Image.open(path) as img:
                    img = img.convert("RGB")
                    arr = np.array(img)
                    hist_r += np.bincount(arr[:,:,0].ravel(), minlength=256)
                    hist_g += np.bincount(arr[:,:,1].ravel(), minlength=256)
                    hist_b += np.bincount(arr[:,:,2].ravel(), minlength=256)
            except Exception as e:
                print(f"⚠️ Error reading {path}: {e}")

    # Normalize for better visual comparison
    hist_r = hist_r / hist_r.sum()
    hist_g = hist_g / hist_g.sum()
    hist_b = hist_b / hist_b.sum()

    # Create intensity axis
    intensities = np.arange(256)

    # Use Seaborn for consistent style
    sns.set(style="whitegrid")
    plt.figure(figsize=(10, 6))
    plt.plot(intensities, hist_r, color='red', label='Red Channel', alpha=0.7)
    plt.plot(intensities, hist_g, color='green', label='Green Channel', alpha=0.7)
    plt.plot(intensities, hist_b, color='blue', label='Blue Channel', alpha=0.7)

    plt.title("Normalized Pixel Intensity Distribution per Channel (Train Images)", fontsize=13)
    plt.xlabel("Pixel Intensity (0–255)")
    plt.ylabel("Normalized Frequency")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
plot_channel_histograms(TRAIN_IMAGE_FOLDER, sample_limit=None, batch_size=16)

In [ ]:
def plotImages(tools,directory):
    print(tools)
    multipleImages = glob(directory)
    plt.rcParams['figure.figsize'] = (8, 8) #Original is 15,15. Since we have 18 veggies I decreased the size
    plt.subplots_adjust(wspace=0, hspace=0)
    i_ = 0
    for l in multipleImages[:25]: #Original is 25
        im = cv2.imread(l)
        im = cv2.resize(im, (256, 256)) 
        plt.subplot(5, 5, i_+1) #.set_title(l)
        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB)); plt.axis('off')
        i_ += 1

plotImages("Biomass train images","../input/csiro-biomass/train/**")